In [1]:
import numpy as np

class SimpleRNN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.hidden_dim = hidden_dim
        self.Wxh = np.random.randn(input_dim, hidden_dim) * 0.01
        self.Whh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Why = np.random.randn(hidden_dim, output_dim) * 0.01
        self.bh = np.zeros((1, hidden_dim))
        self.by = np.zeros((1, output_dim))

    def forward(self, inputs):
        T = inputs.shape[0]
        self.inputs = inputs
        self.hs = {}
        self.hs[-1] = np.zeros((1, self.hidden_dim))
        self.outputs = []

        for t in range(T):
            x_t = inputs[t].reshape(1, -1)
            h_t = np.tanh(x_t.dot(self.Wxh) + self.hs[t-1].dot(self.Whh) + self.bh)
            self.hs[t] = h_t
            y_t = h_t.dot(self.Why) + self.by
            self.outputs.append(y_t)

        self.outputs = np.concatenate(self.outputs, axis=0)
        return self.outputs

    def backward(self, doutputs, learning_rate=0.001):
        T = doutputs.shape[0]
        dWhy = np.zeros_like(self.Why)
        dWhh = np.zeros_like(self.Whh)
        dWxh = np.zeros_like(self.Wxh)
        dby = np.zeros_like(self.by)
        dbh = np.zeros_like(self.bh)
        dh_next = np.zeros((1, self.hidden_dim))

        for t in reversed(range(T)):
            dy = doutputs[t].reshape(1, -1)
            dWhy += self.hs[t].T.dot(dy)
            dby += dy
            dh = dy.dot(self.Why.T) + dh_next
            dtanh = (1 - self.hs[t] ** 2) * dh
            dWxh += self.inputs[t].reshape(-1, 1).dot(dtanh)
            dbh += dtanh
            dWhh += self.hs[t-1].T.dot(dtanh)
            dh_next = dtanh.dot(self.Whh.T)

        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh  -= learning_rate * dbh
        self.by  -= learning_rate * dby

if __name__ == "__main__":
    input_dim = 4
    hidden_dim = 5
    output_dim = 3
    sequence_length = 10

    rnn = SimpleRNN(input_dim, hidden_dim, output_dim)
    inputs = np.random.randn(sequence_length, input_dim)
    outputs = rnn.forward(inputs)
    print("Forward Pass Outputs:")
    print(outputs)

    doutputs = np.ones((sequence_length, output_dim))
    rnn.backward(doutputs, learning_rate=0.01)

    print("\nBackward Pass Completed. Parameters updated.")


Forward Pass Outputs:
[[-2.01600925e-04 -1.07620126e-03  1.33916057e-03]
 [-1.94870677e-04 -3.97884191e-04  4.63720546e-04]
 [ 8.13697322e-05 -6.59966741e-05  8.86773570e-05]
 [ 3.40023009e-04  4.71201408e-04 -3.48488060e-04]
 [ 6.44062211e-04  4.09715651e-04 -1.55816747e-05]
 [-1.80770270e-04 -4.56857358e-04  8.67482200e-04]
 [ 1.12238253e-03  1.70516506e-04  1.37462835e-04]
 [ 6.60920968e-04 -1.99340621e-04  6.08835489e-04]
 [ 2.85096706e-04 -2.75312162e-04  3.38428363e-04]
 [ 4.95014941e-04 -3.94255450e-04  6.68293608e-04]]

Backward Pass Completed. Parameters updated.
